<a href="https://colab.research.google.com/github/Avani1994/NLP/blob/main/Mistral_7B_4bit_QLoRA_Fine_tuning_Explained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install necessary dependencies and import them. We will need

*   accelerate - needed when using bitsandbytes quantization. It's used for efficient distributed training
*   transformers - for AutoTokenizer and AutoModelForCausalLM
*   peft - For adding LORA configuration adopter layer
*   bitsandbytes - For defining quantization config for loading model in 4 bits precision
*   trl - for Supervised Fine Tuning
*   datasets - for loading huggingface dataset
*   wandb - to log your experiments, metrics, memory usage, compute uage


In [1]:
!pip install -q -U accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00


In [2]:
!pip install -q datasets==2.16.0
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [3]:
!pip show transformers

Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch,wandb
from datasets import load_dataset
from trl import SFTTrainer
#HfArgumentParser

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [6]:
# Monitering the LLM
# wandb.login(key = secret_wandb)
# run = wandb.init(
#     project='Fine tuning mistral 7B',
#     job_type="training",
#     anonymous="allow"
# )

Specifiy the base model and the dataset you want to tune with. Here I am using mlabonne/guanaco-llama2-1k dataset.
It contains 1000 samples from guanaco unchained dataset.

You could also use databricks-dolly-15k which is American english dataset created by databricks employees in several behavioral categories outlined in the InstructGPT paper

In [7]:
base_model = "mistralai/Mistral-7B-v0.1"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "mistral_7b_guanaco_avani"

In [8]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="train")
dataset["text"][100]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

'<s>[INST] cuanto es 2x2 xD [/INST] La respuesta es 4. </s><s>[INST] puedes demostrarme matematicamente que 2x2 es 4? [/INST] En una multiplicación, el producto es el resultado de sumar un factor tantas veces como indique el otro, es decir, si tenemos una operación v · n = x, entonces x será igual a v sumado n veces o n sumado v veces, por ejemplo, para la multiplicación 3 · 4 podemos sumar "3 + 3 + 3 + 3" o "4 + 4 + 4" y en ambos casos nos daría como resultado 12, para el caso de 2 · 2 al ser iguales los dos factores el producto sería "2 + 2" que es igual a 4 </s>'

In [9]:
len(dataset)

1000

Here we are using BitsAndBytesConfig to load the based model in 4 bit precision. Some important parameters in this config:

1.   bnb_4bit_quant_type= nf4 or fp4,
- nf4 - Floating Point 4-bit precision
- fp4 - Floating Point 4-bit precision

- fp4 quantizes floats into 4 bits. This video (at the specific timestamp) explains it in more detail: https://youtu.be/TPcXVJ1VSRI?t=563.
- nf4 Constructs a quantization data type where each bin has equal area under a standard normal distribution N(0, 1) that is normalized into the range [-1, 1].


2.   bnb_4bit_compute_dtype = torch.bfloat16, why is it torch.bfloat16
From QLoRA paper:

“QLORA has one low-precision storage data type, in our case usually 4-bit, and one computation data type that is usually BFloat16. In practice, this means whenever a QLORA weight tensor is used, we dequantize the tensor to BFloat16, and then perform a matrix multiplication in 16-bit.for CausalLM models, the last lm_head is kept in its original dtype.”

This simply means that the tensors are stored in 4-bit quantized format as proposed by QLoRa and whenever a computation needs to be performed, these are pulled in the “computation” data type which is usually FP16 or BF16.

Note that in QLoRa (a parameter-efficient fine-tuning technique), the purpose is to freeze the original layers (which are now quantized in 4-bit and only converted to bnb_4bit_compute_dtype when any calculations are to be performed) and only train the new adapter weights in FP32 or in mixed-precision if fp16=True is specified while fine-tuning.


In [10]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

Some important notes when loading model for finetuning
- torch_dtype=torch.bfloat16, Training the model in float16 is not recommended and is known to produce nan; as such, the model should be trained in bfloat16.
- model.config.use_cache = False, When fine-tuning the model, you want to use the updated model params. Using the old (cached) values kinda defeats the purpose of finetuning. Hence, the past (cached) key values are disregarded for the fine-tuned model.
- model.config.pretraining_tp = 1,Setting config.pretraining_tp to a value different than 1 will activate the more accurate but slower computation of the linear layers, which should better match the original logits. But setting to 1 makes it more memory efficient
- model.gradient_checkpointing_enable(), Gradient check-pointing is only needed if training leads to out-of-memory (OOM) errors

In [11]:
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        #load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

We have to fix the tokenizer because in orginal base model training we didnt have a pad token/ eos token, as llms are **decoder** only models and are trained to iterate on your prompt without needing to stop as instructed by pad token or end of sentence token.

But during *intruction fine tuning we need to make sure our sequences are of same length* and hence we need to add pad token to make inputs of same length.

Some references - https://www.reddit.com/r/LocalLLaMA/comments/184g120/mistral_fine_tuning_eos_and_padding/


In [12]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

LORA parameters:
- lora_alpha=16, How much role do LORA wt matrices play while combining with base model params. It acts as a scaling factor, determining how much influence the low-rank matrices have during the fine-tuning process.
- lora_dropout=0.1, adding dropout in LORA layers
- r=64, Rank of LORA weight matrices, larger rank more parameters, better, generated text will be close to expectations. A higher rank matrix means more parameters will be adapted, potentially allowing for a better fit to new data. Conversely, a smaller rank matrix means fewer parameters will change, ensuring more of the pre-trained model’s original structure is preserved.
- bias="none",
- task_type="CAUSAL_LM",
- target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"], modules to focus on while doing LORA fine Tuning.
This is based on the QLoRA paper: "*We find that the most critical LoRA hyperparameter is how many LoRA adapters are used in total and that LoRA on all linear transformer block layers is required to match full finetuning performance.*"


In [13]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

- learning rate warmup - A phase in the beginning of your neural network training where you start with a learning rate much smaller than your "initial" learning rate and then increase it over a few iterations or epochs until it reaches that "initial" learning rate
- lr_scheduler - adjusting the learning rate schedule to go up or down at periodic intervals or at every step or at some steps.
- weight_decay - a regularization technique by adding a small penalty, usually the L2 norm of the weights (all the weights of the model), to the loss function. It is used to reduce overfitting
- gradient_accumulation_steps - batch broken into mini batches, add gradients from each minibatch without updation. Technique to reduce memory requirements while training on GPU

In [14]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)


Supervised Fientuning

In [15]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.034900
50,1.478300
75,1.073700
100,1.352500
125,1.049500
150,1.248500
175,1.046300
200,1.295500
225,1.005300
250,1.347500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=250, training_loss=1.193200309753418, metrics={'train_runtime': 6433.6246, 'train_samples_per_second': 0.155, 'train_steps_per_second': 0.039, 'total_flos': 1.874641569231667e+16, 'train_loss': 1.193200309753418, 'epoch': 1.0})

In [17]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,▂█▁▄▁▄▁▅▁▃
train/learning_rate,▁▁▁▁▁▁▁▁▁▁
train/loss,▁█▂▆▂▅▂▅▁▆
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Line

In [ ]:
# try:
trainer.model.push_to_hub(new_model)#, use_temp_dir=False)
# except:
#     print("An exception occurred")

In [18]:
logging.set_verbosity(logging.CRITICAL)

prompt = "How can I get all knowledge in the field of Machine Learning?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How can I get all knowledge in the field of Machine Learning? [/INST] There are many ways to get knowledge in the field of Machine Learning. You can read books, take online courses, attend conferences, and participate in online communities. You can also try to solve real-world problems using Machine Learning techniques. 

Here are some resources that you can use to get started:

1. Books:
- "Machine Learning: A Probabilistic Perspective" by Kevin Murphy
- "Machine Learning: An Algorithmic Perspective" by Stephen Marsland
- "Machine Learning: A Mathematical Perspective" by Stephen Marsland

2. Online courses:
- Coursera: Machine Learning Specialization
- Udacity: Deep Learning Nanodegree
- edX: Machine Learning MicroMasters

3. Conferences:
- NeurIPS (Neural Information Processing Systems)


In [19]:
prompt = "What is best way for investment"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is best way for investment [/INST] There is no one best way to invest. It depends on your goals, risk tolerance, and time horizon. Some common investment strategies include:

1. Diversification: Investing in a variety of assets, such as stocks, bonds, and real estate, to reduce risk.
2. Value investing: Investing in companies that are undervalued by the market.
3. Growth investing: Investing in companies that are expected to grow rapidly.
4. Index investing: Investing in a fund that tracks a market index, such as the S&P 500.
5. Active investing: Investing in individual stocks and bonds, with the goal of outperforming the market.

It's important to do your research and consult with a financial advisor to determine the best investment strategy for you. 🤓 📈 �


In [20]:
prompt = "For Index investing according to you what is the most profitable fund these days with maximum return"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] For Index investing according to you what is the most profitable fund these days with maximum return [/INST] I am not a financial advisor and I cannot give you advice on what to invest in. 

I can however give you some information on the topic. 

The most profitable fund is the one that has the highest return. 

The most profitable fund is the one that has the highest return. 

The most profitable fund is the one that has the highest return. 

The most profitable fund is the one that has the highest return. 

The most profitable fund is the one that has the highest return. 

The most profitable fund is the one that has the highest return. 

The most profitable fund is the one that has the highest return. 

The most profitable fund is the one that has the highest return. 

The most profitable fund is the one that has the highest


In [22]:
prompt = "You are not a finacial advisor, I understand, I am just looking this information for educational purposes. What do you think for Index investing according to you what is the most profitable fund these days with maximum return."
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] You are not a finacial advisor, I understand, I am just looking this information for educational purposes. What do you think for Index investing according to you what is the most profitable fund these days with maximum return. [/INST] I am not a financial advisor, and I cannot give you advice on what to invest in. 

I can tell you that the S&P 500 has been a good investment over the long term, and that the Vanguard S&P 500 ETF (VOO) is a good way to invest in the S&P 500. 

I can also tell you that the Vanguard Total Stock Market ETF (VTI) is a good way to invest in the entire US stock market. 

I can also tell you that the Vanguard Total World Stock ETF (VT) is a good way to invest in the entire world
